Define a matrix *staggered* if it contains only 0,1,2,3, and there's no equal adjacent (including diagonal) elements in it. 

Question: how to find out all MxN staggered matrix?

When M=N=2, the problem is trivial. One simple solution is:
[[0, 1],   
 [2, 3]]   
To find all solutions, we can just map 0,1,2,3 to other permutations of 0,1,2,3.

Starts from this 2x2 case, gradually expand the matrix.

Make a function `top_pad`, expand any NxN matrix to (N+1)xN. The extra top row only needs to consider the original top row. In that function   

First, fill all "determined" elements, where the three elements under it contain 3 different values.

Then, starts from the left, determine the undefined elements one by one. Each determination would introduce 2 possibilities, meanwhile making some other elements determined. Fill these determined elements.

Recursively fill the elements, until all elements are filled.

Want Nx(N+1) matrix? Just transpose the matrix, top_pad, then transpose back.

In [1]:
import numpy as np

In [2]:
ALL_CHOICES = np.array([0,1,2,3], dtype=np.int)
BASE_MAT = np.array([[0,1],[2,3]], dtype=np.int)

def find_quad_staggered_matrix(M,N, *,
                               base_mat=BASE_MAT):
    mats = [base_mat]
    
    print("Expanding rows...")
    for a in range(2,M):
        print("{} to {}...".format(a,a+1))
        for _ in range(len(mats)):
            mat = mats.pop(0)
            mats += top_pad(mat)
            
    print("Expanding columns...")
    for b in range(2,N):
        print("{} to {}...".format(b,b+1))
        for _ in range(len(mats)):
            mat = mats.pop(0)
            mats += left_pad(mat)
            
    return mats

def top_pad(mat):
    rtv = []
    pad_vecs = find_pad_vecs(mat[0])
    for vec in pad_vecs:
        rtv.append(np.concatenate([np.expand_dims(vec, 0), mat], axis=0))
    return rtv

def left_pad(mat):
    return [np.transpose(m) for m in top_pad(np.transpose(mat))]

def find_pad_vecs(ref):
    """
    ref should be a length-n vector, staggered
    """
    # build extra row
    N = len(ref)
    assert N>=2
    pending_vecs = [np.full_like(ref, -1)] # -1 means undefined elements
    pad_vecs = [] # hold ready-to-pad rows
    
    # iterately find possible pad_vecs
    while len(pending_vecs)>0:
        # fill the elements that can be determined
        pvec = pending_vecs.pop(0)
        pvec, undef_indices = _fill_determined(pvec, ref)
        if pvec is None: # conflicts!
            continue
        
        # if all elements are defined...
        if len(undef_indices)==0: 
            pad_vecs.append(pvec)
            continue
        
        # fork fill the first undefined element, put it back to pending_vecs
        ind = undef_indices[0]
        choices = _check_choices(pvec, ind, ref)
        assert len(choices)==2, "It should only be 2"
        pending_vecs += _fork_fill(pvec, ind, choices)
    
    return pad_vecs

def _fill_determined(vec, ref):
    """
    ref is the row behind vec
    Return:
        vec - filled vector
        undef_indices - the indices of undefined elements, for easier post process
    Returned vec may be None, if there's conflicting elements
    """
    cant_fill_more = False
    while not cant_fill_more:
        cant_fill_more = True # assume that this is the last check
        undef_indices = np.where(vec<0)[0]
        if len(undef_indices)==0: # no undifined elements!
            break
        for ind in undef_indices:
            choices = _check_choices(vec, ind, ref)
            if len(choices)==0: # conflict!
                return None, None
            elif len(choices)==1: # find one element to fill
                vec[ind] = choices[0]
                cant_fill_more = False
                break
            elif len(choices)==2: #needs to fill
                pass
            else:
                raise RuntimeError("Why one element can have more than 2 choices???")
        
    return vec, undef_indices

def _fork_fill(vec, ind, choices):
    """
    Feed in a vector with undefined elements, fill in the specific index
    ref is the row behind vec
    Check that elements surely have 2 choices
    """
    rtv = []
    for c in choices:
        v = np.copy(vec)
        v[ind] = c
        rtv.append(v)
    return rtv

def _check_choices(vec, ind, ref):
    """
    Feed in a vector with undefined elements, fill in the specific index
    ref is the row behind vec
    returns the possible values to make it staggered for the element at ind in vec
    """
    N = len(vec)
    assert vec[ind]==-1, "That element is determined!"
    llim, rlim = max(ind-1, 0), min(ind+2, N)
    choices = np.setdiff1d(ALL_CHOICES, np.concatenate([vec[llim:rlim], ref[llim:rlim]]))
    return choices

In [3]:
AT = find_quad_staggered_matrix(3,3)
print("\nAmount of matrices: {}\n".format(len(AT)))

for m in AT:
    print(m)
    print()

Expanding rows...
2 to 3...
Expanding columns...
2 to 3...

Amount of matrices: 3

[[1 2 3]
 [3 0 1]
 [1 2 3]]

[[3 2 3]
 [1 0 1]
 [3 2 3]]

[[2 3 2]
 [1 0 1]
 [3 2 3]]



In [4]:
AT = find_quad_staggered_matrix(4,4)
print("\nAmount of matrices: {}\n".format(len(AT)))

for m in AT:
    print(m)
    print()

Expanding rows...
2 to 3...
3 to 4...
Expanding columns...
2 to 3...
3 to 4...

Amount of matrices: 7

[[0 1 0 1]
 [2 3 2 3]
 [0 1 0 1]
 [2 3 2 3]]

[[2 1 0 1]
 [0 3 2 3]
 [2 1 0 1]
 [0 3 2 3]]

[[0 3 0 1]
 [2 1 2 3]
 [0 3 0 1]
 [2 1 2 3]]

[[2 3 0 1]
 [0 1 2 3]
 [2 3 0 1]
 [0 1 2 3]]

[[1 0 1 0]
 [2 3 2 3]
 [0 1 0 1]
 [2 3 2 3]]

[[0 1 0 1]
 [3 2 3 2]
 [0 1 0 1]
 [2 3 2 3]]

[[1 0 1 0]
 [3 2 3 2]
 [0 1 0 1]
 [2 3 2 3]]



In [5]:
AT = find_quad_staggered_matrix(5,5)
print("\nAmount of matrices: {}\n".format(len(AT)))

for m in AT:
    print(m)
    print()

Expanding rows...
2 to 3...
3 to 4...
4 to 5...
Expanding columns...
2 to 3...
3 to 4...
4 to 5...

Amount of matrices: 15

[[1 0 1 2 3]
 [3 2 3 0 1]
 [1 0 1 2 3]
 [3 2 3 0 1]
 [1 0 1 2 3]]

[[3 0 1 2 3]
 [1 2 3 0 1]
 [3 0 1 2 3]
 [1 2 3 0 1]
 [3 0 1 2 3]]

[[1 2 1 2 3]
 [3 0 3 0 1]
 [1 2 1 2 3]
 [3 0 3 0 1]
 [1 2 1 2 3]]

[[3 2 1 2 3]
 [1 0 3 0 1]
 [3 2 1 2 3]
 [1 0 3 0 1]
 [3 2 1 2 3]]

[[1 0 3 2 3]
 [3 2 1 0 1]
 [1 0 3 2 3]
 [3 2 1 0 1]
 [1 0 3 2 3]]

[[3 0 3 2 3]
 [1 2 1 0 1]
 [3 0 3 2 3]
 [1 2 1 0 1]
 [3 0 3 2 3]]

[[1 2 3 2 3]
 [3 0 1 0 1]
 [1 2 3 2 3]
 [3 0 1 0 1]
 [1 2 3 2 3]]

[[3 2 3 2 3]
 [1 0 1 0 1]
 [3 2 3 2 3]
 [1 0 1 0 1]
 [3 2 3 2 3]]

[[2 3 2 3 2]
 [1 0 1 0 1]
 [3 2 3 2 3]
 [1 0 1 0 1]
 [3 2 3 2 3]]

[[3 2 3 2 3]
 [0 1 0 1 0]
 [3 2 3 2 3]
 [1 0 1 0 1]
 [3 2 3 2 3]]

[[2 3 2 3 2]
 [0 1 0 1 0]
 [3 2 3 2 3]
 [1 0 1 0 1]
 [3 2 3 2 3]]

[[3 2 3 2 3]
 [1 0 1 0 1]
 [2 3 2 3 2]
 [1 0 1 0 1]
 [3 2 3 2 3]]

[[2 3 2 3 2]
 [1 0 1 0 1]
 [2 3 2 3 2]
 [1 0 1 0 1]
 [3 2 3 2 3]]

[[3 2

In [6]:
AT = find_quad_staggered_matrix(6,6)
print("\nAmount of matrices: {}\n".format(len(AT)))

for m in AT:
    print(m)
    print()

Expanding rows...
2 to 3...
3 to 4...
4 to 5...
5 to 6...
Expanding columns...
2 to 3...
3 to 4...
4 to 5...
5 to 6...

Amount of matrices: 31

[[0 1 0 1 0 1]
 [2 3 2 3 2 3]
 [0 1 0 1 0 1]
 [2 3 2 3 2 3]
 [0 1 0 1 0 1]
 [2 3 2 3 2 3]]

[[2 1 0 1 0 1]
 [0 3 2 3 2 3]
 [2 1 0 1 0 1]
 [0 3 2 3 2 3]
 [2 1 0 1 0 1]
 [0 3 2 3 2 3]]

[[0 3 0 1 0 1]
 [2 1 2 3 2 3]
 [0 3 0 1 0 1]
 [2 1 2 3 2 3]
 [0 3 0 1 0 1]
 [2 1 2 3 2 3]]

[[2 3 0 1 0 1]
 [0 1 2 3 2 3]
 [2 3 0 1 0 1]
 [0 1 2 3 2 3]
 [2 3 0 1 0 1]
 [0 1 2 3 2 3]]

[[0 1 2 1 0 1]
 [2 3 0 3 2 3]
 [0 1 2 1 0 1]
 [2 3 0 3 2 3]
 [0 1 2 1 0 1]
 [2 3 0 3 2 3]]

[[2 1 2 1 0 1]
 [0 3 0 3 2 3]
 [2 1 2 1 0 1]
 [0 3 0 3 2 3]
 [2 1 2 1 0 1]
 [0 3 0 3 2 3]]

[[0 3 2 1 0 1]
 [2 1 0 3 2 3]
 [0 3 2 1 0 1]
 [2 1 0 3 2 3]
 [0 3 2 1 0 1]
 [2 1 0 3 2 3]]

[[2 3 2 1 0 1]
 [0 1 0 3 2 3]
 [2 3 2 1 0 1]
 [0 1 0 3 2 3]
 [2 3 2 1 0 1]
 [0 1 0 3 2 3]]

[[0 1 0 3 0 1]
 [2 3 2 1 2 3]
 [0 1 0 3 0 1]
 [2 3 2 1 2 3]
 [0 1 0 3 0 1]
 [2 3 2 1 2 3]]

[[2 1 0 3 0 1]
 [0 3 2 1 2 3

Guess: there's $4! \times (2^{N-1}-1)$staggered matrix with NxN elements

In [7]:
AT = find_quad_staggered_matrix(4,4, base_mat=np.array([[0,1],[3,2]],dtype=np.int))
print("\nAmount of matrices: {}\n".format(len(AT)))

for m in AT:
    print(m)
    print()

Expanding rows...
2 to 3...
3 to 4...
Expanding columns...
2 to 3...
3 to 4...

Amount of matrices: 7

[[0 1 0 1]
 [2 3 2 3]
 [0 1 0 1]
 [3 2 3 2]]

[[1 0 1 0]
 [2 3 2 3]
 [0 1 0 1]
 [3 2 3 2]]

[[0 1 0 1]
 [3 2 3 2]
 [0 1 0 1]
 [3 2 3 2]]

[[3 1 0 1]
 [0 2 3 2]
 [3 1 0 1]
 [0 2 3 2]]

[[0 2 0 1]
 [3 1 3 2]
 [0 2 0 1]
 [3 1 3 2]]

[[3 2 0 1]
 [0 1 3 2]
 [3 2 0 1]
 [0 1 3 2]]

[[1 0 1 0]
 [3 2 3 2]
 [0 1 0 1]
 [3 2 3 2]]



In [ ]:
[[0 1 0 1]
 [3 2 3 2]
 [0 1 0 1]
 [3 2 3 2]]